In [3]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import requests
import time

In [24]:
import csv
with open('../indice/ura_data_processed.csv', mode='r', encoding='utf-8') as file:
    csv_reader = csv.DictReader(file)

dataset = pd.read_csv('../indice/ura_data_processed.csv', index_col=0)

In [46]:
bool_cols = dataset.select_dtypes(include='bool').columns
object_cols = dataset.select_dtypes(include='object').columns
dataset[bool_cols] = dataset[bool_cols].astype(int)
dataset.dtypes
object_cols

Index(['Building_age'], dtype='object')

In [49]:
if __name__ == '__main__':
    y = dataset['log_price_psf']
    X_columns = [c for c in dataset.columns if not c.startswith('Period_') and 
                 not c in ['log_price_psf'] and not c in ['Building_age']]
    X = dataset[X_columns]
    X = sm.add_constant(X)

    model = sm.OLS(y, X)
    results = model.fit()
    print(results.summary())



                            OLS Regression Results                            
Dep. Variable:          log_price_psf   R-squared:                       0.580
Model:                            OLS   Adj. R-squared:                  0.580
Method:                 Least Squares   F-statistic:                     2587.
Date:                Mon, 25 Mar 2024   Prob (F-statistic):               0.00
Time:                        01:55:08   Log-Likelihood:                 6594.9
No. Observations:               28064   AIC:                        -1.316e+04
Df Residuals:                   28048   BIC:                        -1.303e+04
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 5.9350      0.03